## A Simulated IOT Federated Learning Framework for Forest Fire Prediction

Forest fire ignition prediction is essential to the safety of communities and the proper allocation of fire fighting resources. This notebook simulates a network of clients that collect local meteorological data. Each client participates in a federated learning system where a client trains a machine learning model to classify whether the area will have a forest fire ignition. The model weights of each client are shared with a central server that averages the weights and shares them back to the clients.

This notebook demonstrates the two machine learning types shown below.

|<center>name|<center>server|<center>clients|<center>training|<center>training data|<center>evaluation type|<center>evaluation|<center>evaluation data|
|---|---|---|---|---|---|---|---|
|<center>central server ML|<center>yes|<center>no|<center>on server|<center>on server|<center>centralized|<center>on server|<center>on server|
|<center>federated ML (Federated Eval)|<center>yes|<center>yes|<center>on clients|<center>on clients|<center>federated|<center>on clients|<center>on clients|


The data used is from the paper: *Framework for Creating Forest Fire Ignition Prediction Datasets.* Each row represents meteorological data at a geographical location at a specific time. TODO: Add table example.

Much of the code used in this notebook is based on the Flower code examples located [here](https://github.com/adap/flower/tree/main/examples) and the Keras timeseries tutorials located [here](https://keras.io/examples/timeseries/).
The code below is very much a work in progress. 

In [1]:
#if this file is being used in colab set to 1 otherwise 0
using_colab = 1

In [2]:
if (using_colab == 1):
    from google.colab import drive
    drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
if (using_colab ==1):
    from psutil import virtual_memory
    ram_gb = virtual_memory().total / 1e9
    print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

    if ram_gb < 20:
        print('Not using a high-RAM runtime')
    else:
        print('You are using a high-RAM runtime!')

Your runtime has 54.8 gigabytes of available RAM

You are using a high-RAM runtime!


In [4]:
#load libraries
import math
import os
import glob
import gc
import datetime
import typing
from typing import List
from typing import Tuple
from typing import Dict
from typing import Optional
import random
import tempfile
from tables.file import File

import pandas as pd
import numpy as np

import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras import Model

In [5]:
if (using_colab == 1):
    !pip install -q flwr[simulation] pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.2/157.2 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.2/56.2 MB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 91.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 57.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.4/201.4 kB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 81.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.9/97.9 kB 11.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 82.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 13.8

In [6]:
import flwr as fl
from flwr.common import Metrics

In [7]:
#overall environment settings

# Make TensorFlow logs less verbose
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

# Training on GPU or CPU?
#tf.config.set_visible_devices([], 'CPU')
python_version = !python --version
print(
    f"Training on {'GPU' if tf.config.get_visible_devices('GPU') else 'CPU'}\
    using TensorFlow {tf.__version__}, Flower {fl.__version__} and {python_version[0]}"
)

Training on CPU    using TensorFlow 2.12.0, Flower 1.4.0 and Python 3.10.12


In [8]:
#global variables

ml_type = 0 # classic ML = 0, federated ML w/ centralized evaluation = 1, federated ML w/ federated eval = 2

cid = str(0) # preliminary client id

master_path = <path to your master directory for this simulation>
federated_path = <path to your client datasets> 
centralized_path = <path to your server dataset
results_path = <path to where you want to store results>
temp_path = master_path + "history_temp/"
log_dir = <path to where you want to store Tensorflow logs> + cid + "_" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S") 

downsample_test_set = 0 # 0 if test set not downsampled, 1 otherwise

# TODO: Add the below to a client config file
NUM_CLIENTS = 24
num_rounds = 4
epochs = 10 # the number of epochs classic machine learning will use
fl_epochs = 3

sequence_len = 120 # 5 days * 24 hours
past_len = sequence_len
future_len = 24 # 1*24 hours
sampling_rate = 1 # in time series conversion use every row (hour) in loaded datasets
sequence_stride = 1 # in time series conversion each series is this far apart

FRACTION_FIT=0.75,  # Sample X% of available clients for training
FRACTION_EVALUATE=0.8,  # Sample X% of available clients for evaluation
MIN_FIT_CLIENTS=13,  # Never sample less than X clients for training
MIN_EVALUATE_CLIENTS=12,  # Never sample less than X clients for evaluation
MIN_AVAILABLE_CLIENTS=13,  # Wait until at least X clients are available

#create an array for final results
np.set_printoptions(suppress=True)
results = np.empty([13,2])

#create an array for interim results
metric_names = ['tp', 'fp', 'tn', 'fn', 'accuracy', 'precision', 'recall', 'auc', 'prc']
metric_names_len = len(metric_names)
interim_results = np.zeros((NUM_CLIENTS, metric_names_len, num_rounds))

## Data loading functions

In [9]:
def normalize_data(x):
    """Normalizes the data of an array by column. 
    Shifts and scales inputs into a distribution centered around 0 
    with standard deviation 1.

    Parameters
    ----------
    x: NDarray 
        An array of feature values.
        
    Returns
    -------
    features_normalized : NDarray
        The original array, but normalized.
    """
    data = x
    layer = layers.Normalization()
    layer.adapt(data)
    features_normalized = layer(data)
    return features_normalized

In [10]:
def mask_create(x):
    """Finds the class count of the input array and creates a mask that can be used
    to randomly downsample an array of labels so that the number of 
    negative labels = the number of positive labels. 

    Parameters
    ----------
    x: NDarray 
        An array of feature values.
        
    Returns
    -------
    features_normalized : NDarray
        A masked version of the input array.
    """
    mask_length = x.shape[0]
    mask = tf.reshape(x, [mask_length])
    y, idx, class_count = tf.unique_with_counts(mask)
    ignition_count = tf.get_static_value(class_count[1])
    mask = mask.numpy()
    count = 0 
    while count < ignition_count:
        #rand_num = random.randint(0,mask_length)
        rand_num = random.randint(1, mask_length-1)
        if (mask[rand_num] == 0):
            mask[rand_num] = 1
            count += 1
    return mask

In [11]:
def load_datasets(path: str):
    """Loads all the csv datasets in a folder.
    The loaded data is divided into train, validation, and test sets.
    The data is turned into time series data
    All the data is normalized.
    Train and validation datasets are downsampled.
    TODO: divide this function into smaller functions

    Parameters
    ----------
    path: string 
        The path to the dataset folder.
        
    Returns
    -------
    train_x, train_y, val_x, val_y, test_x, test_y : NDarrays
        A masked version of the input array.
    """

    train_x = []
    train_y = []
    val_x = []
    val_y = []
    test_x = []
    test_y = []

    #load data
    for filename in glob.glob(os.path.join(path, '*.csv')):
        print("\nnow reading " + filename + "\n")
        #read file
        df = pd.read_csv(filename, index_col=[0])
        
        df_train = df[(df['year'] < 2001)]
        df_val = df[(df['year'] > 2001) & (df['year'] < 2012)]
        df_test = df[(df['year'] >= 2012)]
        
        features = ['stl2',
                    't2m',
                    'stl1',
                    'stl3',
                    'skt',
                    'swvl1', 
                    'd2m',
                    'swvl2'
                    ] #this shoudn't be hard coded
        train_features = df_train[features]
        train_labels = df_train[["ignition"]]
        val_features = df_val[features]
        val_labels = df_val[["ignition"]]
        test_features = df_test[features]
        test_labels = df_test[["ignition"]]
        #convert to numpy
        train_features = train_features.values
        val_features = val_features.values
        test_features = test_features.values

        #normalize
        train_features_normalize = normalize_data(train_features)
        val_features_normalize = normalize_data(val_features)
        test_features_normalize = normalize_data(test_features)
        
        #we want to predict at a future point
        #so we clip the length of the features plus the hours till the future point
        start = past_len + future_len
        train_labels = train_labels.iloc[start:].values
        val_labels = val_labels.iloc[start:].values
        test_labels = test_labels.iloc[start:].values
        
        batch_size = 107856 #factor of 5136 (321 * 16)
        #convert to time series data
        train_dataset = keras.utils.timeseries_dataset_from_array(
            train_features_normalize,
            train_labels,
            sampling_rate=sampling_rate,
            sequence_length=sequence_len,
            sequence_stride = sequence_stride,
            shuffle=False,
            batch_size=batch_size)

        val_dataset = keras.utils.timeseries_dataset_from_array(
            val_features_normalize,
            val_labels,
            sampling_rate=sampling_rate,
            sequence_length=sequence_len,
            sequence_stride = sequence_stride,
            shuffle=False,
            batch_size=batch_size)

        test_dataset = keras.utils.timeseries_dataset_from_array(
            test_features_normalize,
            test_labels,
            sampling_rate=sampling_rate,
            sequence_length=sequence_len,
            sequence_stride = sequence_stride,
            shuffle=False,
            batch_size=batch_size)
        
        #for bookkeeping print out the shapes of the datasets
        for train_features, train_labels in train_dataset:
            print("train_dataset features shape:", train_features.shape)
            print("targets_dataset labels shape:", train_labels.shape)
            break

        for val_features, val_labels in val_dataset:
            print("\nval_dataset features shape:", val_features.shape)
            print("val_dataset labels shape:", val_labels.shape)
            break

        for test_features, test_labels in test_dataset:
            print("\ntest_dataset features shape:", test_features.shape)
            print("test_dataset labels shape:", test_labels.shape)
            break
       
        # randomly downsample the data using masks
        train_mask = mask_create(train_labels)
        train_features_masked = tf.boolean_mask(train_features, train_mask)
        train_labels_masked = tf.boolean_mask(train_labels, train_mask)
        
        val_mask = mask_create(val_labels)
        val_features_masked = tf.boolean_mask(val_features, val_mask)
        val_labels_masked = tf.boolean_mask(val_labels, val_mask)
        
        test_mask = mask_create(test_labels)
        test_features_masked = tf.boolean_mask(test_features, test_mask)
        test_labels_masked = tf.boolean_mask(test_labels, test_mask)
        
        train_x.append(train_features_masked)
        train_y.append(train_labels_masked)
        val_x.append(val_features_masked)
        val_y.append(val_labels_masked)
        if (downsample_test_set == 1):
            test_x.append(test_features_masked)
            test_y.append(test_labels_masked)
        else:
            test_x.append(test_features)
            test_y.append(test_labels)
        
    print("\nDone loading data.\n")
    return train_x, train_y, val_x, val_y, test_x, test_y 



In [12]:
def get_value_count(x):
    """A helper function that returns the count of class labels.

    Parameters
    ----------
    x: NDArray 
        An array with class labels.
        
    Returns
    -------
    non_ignition_count, ignition_count : int
        The counts of the ignition class.
    """
    length = x[0].shape[0]
    x = tf.reshape(x, [length])
    y, idx, class_count = tf.unique_with_counts(x)
    non_ignition_count = tf.get_static_value(class_count[0])
    ignition_count = tf.get_static_value(class_count[1])
    return non_ignition_count, ignition_count
    

## Model and Metrics

In [13]:
METRICS = [
    keras.metrics.TruePositives(name='tp'),
    keras.metrics.FalsePositives(name='fp'),
    keras.metrics.TrueNegatives(name='tn'),
    keras.metrics.FalseNegatives(name='fn'), 
    keras.metrics.BinaryAccuracy(name='accuracy'),
    keras.metrics.Precision(name='precision'),
    keras.metrics.Recall(name='recall'),
    keras.metrics.AUC(name='auc'),
    keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
    #keras.metrics.F1Score(name='f1_score'),#only available with nightly build
]

def make_model(metrics=METRICS):
    inputs = keras.Input(shape=(sequence_len, trainloaders_x[0].shape[2]))
    x = layers.LSTM(8, activation='sigmoid')(inputs)
    x = layers.Flatten()(x)
    outputs = layers.Dense(1, activation="sigmoid")(x)
    model = keras.Model(inputs, outputs)

    model.compile(
        optimizer=keras.optimizers.legacy.Adam(learning_rate=1e-3),
        loss=keras.losses.BinaryCrossentropy(),
        metrics=metrics)
        
    return model


## Federated Learning Functions

In [14]:
class FlowerClient(fl.client.NumPyClient):
    """Instantiates a FlowerClient object

    Parameters
    ----------
    fl.client: NumPyClient
        A premade helper client.
    """
    def __init__(self, cid, model, x_train, y_train, x_val, y_val, x_test, y_test, tb_callback) -> None:
        self.cid = cid
        self.model = model
        self.x_train, self.y_train = x_train, y_train
        self.x_val, self.y_val = x_val, y_val
        self.x_test, self.y_test = x_test, y_test
        self.tb_callback = tb_callback

    def get_parameters(self, config):
        return self.model.get_weights()

    # presetting config allows us to use FlowerClient with classic ml.
    def fit(self, parameters, config = "ServerConfig(num_rounds=5, round_timeout=None)"):
        print("in fit")
        server_round = config["server_round"]
        print("in server round: ", server_round)
        self.model.set_weights(parameters)
        self.model.fit(self.x_train, self.y_train, epochs=epochs, verbose=2, callbacks=self.tb_callback, validation_data=(self.x_val, self.y_val),)
        return self.model.get_weights(), len(self.x_train), {}

    def evaluate(self, parameters, config = "ServerConfig(num_rounds=5, round_timeout=None)"):
        server_round = int(config["server_round"])
        print(f"Hi, this is base station {self.cid} in eval for round {server_round}")
        self.model.set_weights(parameters)
        loss, tp, fp, tn, fn, accuracy, precision, recall, auc, prc  = self.model.evaluate(self.x_test, self.y_test, verbose=2)

        

        # TODO: store the client round data somewhere internally
        # This section writes the client data to a file.
        # It is temporary until I can write a custom client function.

        client_round_results = np.zeros(11)
        
        print("\nWriting out interim results.")
        #write to interim results
        client_round_results[0] = self.cid
        client_round_results[1] = server_round
        client_round_results[2] = tp
        client_round_results[3] = fp
        client_round_results[4] = tn
        client_round_results[5] = fn
        client_round_results[6] = accuracy
        client_round_results[7] = precision
        client_round_results[8] = recall
        client_round_results[9] = auc
        client_round_results[10] = prc

        file = temp_path + str(self.cid) + "_" + str(server_round) + "_" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S") + ".csv"
        pd.DataFrame(client_round_results).to_csv(file)

        return loss, len(self.x_val), {"tp": tp,
                                       "fp": fp,
                                       "tn": tn,
                                       "fn": fn,
                                       "accuracy": accuracy,
                                       "precision": precision,
                                       "recall": recall,
                                       "auc": auc,
                                       "prc": prc
                                      }

    # this method allows metrics to be passed when using a single server with no clients
    def evaluate_single(self, x_test, y_test):
        loss, tp, fp, tn, fn, accuracy, precision, recall, auc, prc = self.model.evaluate(x_test, y_test, verbose=2)
        return loss, tp, fp, tn, fn, accuracy, precision, recall, auc, prc

In [15]:
def client_fn(cid: str) -> fl.client.Client:
    """Instantiates the client object. 

    Parameters
    ----------
    cid: str 
        The client id.
        
    Returns
    -------
    cid : NDarray
    model: model
    test/val/test datasets: NDarray
    tensorboard: tensorflow callbacks
        Everything a client needs to create a ML model
    """

    print("\nThis is client: ", cid)

    x_train_cid = trainloaders_x[int(cid)]
    y_train_cid = trainloaders_y[int(cid)]
    x_val_cid = valloaders_x[int(cid)]
    y_val_cid = valloaders_y[int(cid)]
    x_test_cid = testloaders_x[int(cid)]
    y_test_cid = testloaders_y[int(cid)]

    print("Loaded data for client: ", cid, "\n")

    METRICS = [
        keras.metrics.TruePositives(name='tp'),
        keras.metrics.FalsePositives(name='fp'),
        keras.metrics.TrueNegatives(name='tn'),
        keras.metrics.FalseNegatives(name='fn'), 
        keras.metrics.BinaryAccuracy(name='accuracy'),
        keras.metrics.Precision(name='precision'),
        keras.metrics.Recall(name='recall'),
        keras.metrics.AUC(name='auc'),
        keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
        #keras.metrics.F1Score(name='f1_score'),#only available with nightly build
    ]

    def make_model(metrics=METRICS):
        inputs = keras.Input(shape=(sequence_len, x_train_cid.shape[2]))
        x = layers.LSTM(8, activation='sigmoid')(inputs)
        x = layers.Flatten()(x)
        outputs = layers.Dense(1, activation="sigmoid")(x)
        model = keras.Model(inputs, outputs)

        model.compile(
            optimizer=keras.optimizers.legacy.Adam(learning_rate=1e-3),
            loss=keras.losses.BinaryCrossentropy(),
            metrics=metrics)
        
        return model
    
    print("Making model: ", cid)
    model = make_model()
    
    tensorboard = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

    tb_callback = keras.callbacks.TensorBoard(log_dir="logs/", histogram_freq=1)

    # Create and return client
    print("\nClient CID: " + str(cid) + " is done.\n")
    return FlowerClient(cid, model, x_train_cid, y_train_cid, x_val_cid, y_val_cid, x_test_cid, y_test_cid, tensorboard)

In [16]:
# TODO: this function isn't working correctly, have a look at it
def weighted_average(metrics: List[Tuple[int, Metrics]]) -> Metrics:
    """A function for the final evaluation of the simulation. 
    All metrics are averaged and output.

    Parameters
    ----------
    metrics: List 
        A list of the metrics being used
        
    Returns
    -------
     : Dict
        A dictionary of the averaged results from each round.
    """
    # Multiply each metric of each client by number of examples used
    tps = [num_examples * m["tp"] for num_examples, m in metrics]
    fps = [num_examples * m["fp"] for num_examples, m in metrics]
    tns = [num_examples * m["tn"] for num_examples, m in metrics]
    fns = [num_examples * m["fn"] for num_examples, m in metrics]
    accuracies = [num_examples * m["accuracy"] for num_examples, m in metrics]
    precisions = [num_examples * m["precision"] for num_examples, m in metrics]
    recalls = [num_examples * m["recall"] for num_examples, m in metrics]
    aucs = [num_examples * m["auc"] for num_examples, m in metrics]
    prcs = [num_examples * m["prc"] for num_examples, m in metrics]
    examples = [num_examples for num_examples, _ in metrics]

    # Aggregate and return custom metric (weighted average)
    return {"tp": sum(tps) / sum(examples),
            "fp": sum(fps) / sum(examples),
            "tn": sum(tns) / sum(examples),
            "fn": sum(fns) / sum(examples),
            "accuracy": sum(accuracies) / sum(examples),
            "precision": sum(precisions) / sum(examples),
            "recall": sum(recalls) / sum(examples),
            "auc": sum(aucs) / sum(examples),
            "prc": sum(prcs) / sum(examples)
           }

In [17]:
def fit_config(server_round: int):
    """Passes information from the server to client.

    Parameters
    ----------
    server_round: int
        What server round simulation is in.
        
    Returns
    -------
    config : Dict
        A dictionary of values.
    """
    config = {
        "server_round": server_round,  # The current round of federated learning
        #"local_epochs": 1 if server_round < 2 else 2,  #
    }
    return config

## Launch classic machine learning

In [18]:
# load the dataset for centralized evaluation (for classic ml training and testing) 
trainloaders_x, trainloaders_y, valloaders_x, valloaders_y, testloaders_x, testloaders_y = load_datasets(centralized_path)


now reading /content/drive/MyDrive/Colab Notebooks/FF/data/01_clients/dly_avg_1of1_50.csv

train_dataset features shape: (107712, 120, 8)
targets_dataset labels shape: (107712, 1)

val_dataset features shape: (51216, 120, 8)
val_dataset labels shape: (51216, 1)

test_dataset features shape: (46080, 120, 8)
test_dataset labels shape: (46080, 1)

Done loading data.



In [19]:
count = get_value_count(trainloaders_y)
print("Train set nonignitions and ignitions are:", count)
count = get_value_count(valloaders_y)
print("Validation set nonignitions and ignitions are:", count)
count = get_value_count(testloaders_y)
print("Test set nonignitions and ignitions are:", count)

Train set nonignitions and ignitions are: (2811, 2811)
Validation set nonignitions and ignitions are: (2534, 2534)
Test set nonignitions and ignitions are: (44719, 1361)


In [20]:
# create an initial model to prime flower client
model = make_model()

In [21]:
# train single server classic ml
# create a flower client that represents a classic ml single server
central_server_model = client_fn(str(0))
# set parameters for classic_ml so it can use single flower client function
parameters = model.get_weights()
# run fit
config = fit_config(0)
history = central_server_model.fit(parameters, config)


This is client:  0
Loaded data for client:  0 

Making model:  0

Client CID: 0 is done.

in fit
in server round:  0
Epoch 1/10
176/176 - 12s - loss: 0.6747 - tp: 2499.0000 - fp: 2120.0000 - tn: 691.0000 - fn: 312.0000 - accuracy: 0.5674 - precision: 0.5410 - recall: 0.8890 - auc: 0.6651 - prc: 0.6532 - val_loss: 0.6339 - val_tp: 1845.0000 - val_fp: 929.0000 - val_tn: 1605.0000 - val_fn: 689.0000 - val_accuracy: 0.6807 - val_precision: 0.6651 - val_recall: 0.7281 - val_auc: 0.7144 - val_prc: 0.6851 - 12s/epoch - 68ms/step
Epoch 2/10
176/176 - 8s - loss: 0.6220 - tp: 1908.0000 - fp: 945.0000 - tn: 1866.0000 - fn: 903.0000 - accuracy: 0.6713 - precision: 0.6688 - recall: 0.6788 - auc: 0.7175 - prc: 0.7120 - val_loss: 0.6150 - val_tp: 1743.0000 - val_fp: 783.0000 - val_tn: 1751.0000 - val_fn: 791.0000 - val_accuracy: 0.6894 - val_precision: 0.6900 - val_recall: 0.6878 - val_auc: 0.7221 - val_prc: 0.6979 - 8s/epoch - 45ms/step
Epoch 3/10
176/176 - 8s - loss: 0.6125 - tp: 1864.0000 - fp: 8

In [22]:
#evaluate single server classic ml
loss, tp, fp, tn, fn, accuracy, precision, recall, auc, prc = central_server_model.evaluate_single(testloaders_x[0], testloaders_y[0])

1440/1440 - 15s - loss: 0.5623 - tp: 876.0000 - fp: 10761.0000 - tn: 33958.0000 - fn: 485.0000 - accuracy: 0.7559 - precision: 0.0753 - recall: 0.6436 - auc: 0.7664 - prc: 0.0888 - 15s/epoch - 10ms/step


In [23]:
print("Single server classic ml evaluation\n\
\ttp:\t%d\n\
\tfp:\t%d\n\
\ttn:\t%d\n\
\tfn:\t%d\n\n\
\tloss:\t%f\n\
\tacc:\t%f\n\
\tprec:\t%f\n\
\trec:\t%f\n\
\tauc:\t%f\n\
\tprc:\t%f\
" % (tp,fp,tn,fn,loss,accuracy,precision,recall,auc,prc))

Single server classic ml evaluation
	tp:	876
	fp:	10761
	tn:	33958
	fn:	485

	loss:	0.562342
	acc:	0.755946
	prec:	0.075277
	rec:	0.643644
	auc:	0.766392
	prc:	0.088764


In [24]:
#write to results array
results[0] = tp
results[1][0] = fp
results[2][0] = tn
results[3][0] = fn
results[4][0] = accuracy
results[5][0] = precision
results[6][0] = recall
results[7][0] = auc
results[8][0] = prc
results[9][0] = epochs
results[10][0] = 1
results[11][0] = 1
results[12][0] = 1

In [25]:
#remove data from memory that isn't needed
for_removal = [trainloaders_x, trainloaders_y, valloaders_x, valloaders_y, testloaders_x, testloaders_y]
del(for_removal)
del(model)
del(central_server_model)
gc.collect()
print('Central server data removed.')

Central server data removed.


## Federated machine learning

The Flower federated learning framework is used from here down. More info can be found at https://flower.dev

In [26]:
#TODO: there is a random out-of-bounds error with masking, for now if it occurs run this cell again
#load the dataset for federated learning
trainloaders_x, trainloaders_y, valloaders_x, valloaders_y, testloaders_x, testloaders_y = load_datasets(federated_path)


now reading /content/drive/MyDrive/Colab Notebooks/FF/data/24_clients/121.625_51.75_cell.csv

train_dataset features shape: (107712, 120, 8)
targets_dataset labels shape: (107712, 1)

val_dataset features shape: (51216, 120, 8)
val_dataset labels shape: (51216, 1)

test_dataset features shape: (46080, 120, 8)
test_dataset labels shape: (46080, 1)

now reading /content/drive/MyDrive/Colab Notebooks/FF/data/24_clients/120.625_51.75_cell.csv

train_dataset features shape: (107712, 120, 8)
targets_dataset labels shape: (107712, 1)

val_dataset features shape: (51216, 120, 8)
val_dataset labels shape: (51216, 1)

test_dataset features shape: (46080, 120, 8)
test_dataset labels shape: (46080, 1)

now reading /content/drive/MyDrive/Colab Notebooks/FF/data/24_clients/119.625_51.75_cell.csv

train_dataset features shape: (107712, 120, 8)
targets_dataset labels shape: (107712, 1)

val_dataset features shape: (51216, 120, 8)
val_dataset labels shape: (51216, 1)

test_dataset features shape: (460

In [27]:
#change the number of epochs from single server to federated
epochs = fl_epochs

# Create FedAvg strategy
strategy = fl.server.strategy.FedAvg(
    fraction_fit=FRACTION_FIT[0],  # Sample 10% of available clients for training
    fraction_evaluate=FRACTION_EVALUATE[0],  # Sample 5% of available clients for evaluation
    min_fit_clients=MIN_FIT_CLIENTS[0],  # Never sample less than 10 clients for training
    min_evaluate_clients=MIN_EVALUATE_CLIENTS[0],  # Never sample less than 5 clients for evaluation
    min_available_clients=MIN_AVAILABLE_CLIENTS[0],  # Wait until at least 75 clients are available
    evaluate_metrics_aggregation_fn=weighted_average,  # <-- pass the metric aggregation function
    on_fit_config_fn=fit_config, # Pass the fit_config function
    on_evaluate_config_fn=fit_config # Pass the eval_config function
)

# Start simulation
fl_history = fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=num_rounds),
    strategy=strategy,
)


INFO flwr 2023-06-11 21:02:51,304 | app.py:146 | Starting Flower simulation, config: ServerConfig(num_rounds=4, round_timeout=None)
INFO:flwr:Starting Flower simulation, config: ServerConfig(num_rounds=4, round_timeout=None)
2023-06-11 21:02:54,494	INFO worker.py:1636 -- Started a local Ray instance.
INFO flwr 2023-06-11 21:02:55,829 | app.py:180 | Flower VCE: Ray initialized with resources: {'node:172.28.0.12': 1.0, 'CPU': 8.0, 'object_store_memory': 16231507968.0, 'memory': 32463015936.0}
INFO:flwr:Flower VCE: Ray initialized with resources: {'node:172.28.0.12': 1.0, 'CPU': 8.0, 'object_store_memory': 16231507968.0, 'memory': 32463015936.0}
INFO flwr 2023-06-11 21:02:55,833 | server.py:86 | Initializing global parameters
INFO:flwr:Initializing global parameters
INFO flwr 2023-06-11 21:02:55,836 | server.py:273 | Requesting initial parameters from one random client
INFO:flwr:Requesting initial parameters from one random client
INFO flwr 2023-06-11 21:03:03,029 | server.py:277 | Receiv

(launch_and_get_parameters pid=12578) 
(launch_and_get_parameters pid=12578) This is client:  7
(launch_and_get_parameters pid=12578) Loaded data for client:  7 
(launch_and_get_parameters pid=12578) 
(launch_and_get_parameters pid=12578) Making model:  7
(launch_and_get_parameters pid=12578) 
(launch_and_get_parameters pid=12578) Client CID: 7 is done.
(launch_and_get_parameters pid=12578) 
(launch_and_fit pid=12578) 
(launch_and_fit pid=12578) This is client:  8
(launch_and_fit pid=12578) Loaded data for client:  8 
(launch_and_fit pid=12578) 
(launch_and_fit pid=12578) Making model:  8
(launch_and_fit pid=12578) 
(launch_and_fit pid=12578) Client CID: 8 is done.
(launch_and_fit pid=12578) 
(launch_and_fit pid=12578) in fit
(launch_and_fit pid=12578) in server round:  1
(launch_and_fit pid=12578) Epoch 1/3
(launch_and_fit pid=12578) 66/66 - 13s - loss: 0.7011 - tp: 1053.0000 - fp: 1050.0000 - tn: 3.0000 - fn: 0.0000e+00 - accuracy: 0.5014 - precision: 0.5007 - recall: 1.0000 - auc: 0

DEBUG flwr 2023-06-11 21:09:00,609 | server.py:232 | fit_round 1 received 18 results and 0 failures
DEBUG:flwr:fit_round 1 received 18 results and 0 failures
WARNING flwr 2023-06-11 21:09:00,674 | fedavg.py:243 | No fit_metrics_aggregation_fn provided
DEBUG flwr 2023-06-11 21:09:00,677 | server.py:168 | evaluate_round 1: strategy sampled 19 clients (out of 24)
DEBUG:flwr:evaluate_round 1: strategy sampled 19 clients (out of 24)


(launch_and_fit pid=12578) 51/51 - 3s - loss: 0.6273 - tp: 701.0000 - fp: 454.0000 - tn: 347.0000 - fn: 100.0000 - accuracy: 0.6542 - precision: 0.6069 - recall: 0.8752 - auc: 0.7708 - prc: 0.7710 - val_loss: 0.6071 - val_tp: 566.0000 - val_fp: 291.0000 - val_tn: 331.0000 - val_fn: 56.0000 - val_accuracy: 0.7211 - val_precision: 0.6604 - val_recall: 0.9100 - val_auc: 0.7983 - val_prc: 0.7449 - 3s/epoch - 50ms/step [repeated 2x across cluster]
(launch_and_fit pid=12578) 51/51 - 3s - loss: 0.6273 - tp: 701.0000 - fp: 454.0000 - tn: 347.0000 - fn: 100.0000 - accuracy: 0.6542 - precision: 0.6069 - recall: 0.8752 - auc: 0.7708 - prc: 0.7710 - val_loss: 0.6071 - val_tp: 566.0000 - val_fp: 291.0000 - val_tn: 331.0000 - val_fn: 56.0000 - val_accuracy: 0.7211 - val_precision: 0.6604 - val_recall: 0.9100 - val_auc: 0.7983 - val_prc: 0.7449 - 3s/epoch - 50ms/step
(launch_and_evaluate pid=12578) 
(launch_and_evaluate pid=12578) This is client:  0
(launch_and_evaluate pid=12578) Loaded data for cli

(raylet) Spilled 4322 MiB, 1 objects, write throughput 76 MiB/s. Set RAY_verbose_spill_logs=0 to disable this message.


...

DEBUG flwr 2023-06-11 21:29:53,802 | server.py:182 | evaluate_round 3 received 19 results and 0 failures
DEBUG:flwr:evaluate_round 3 received 19 results and 0 failures
DEBUG flwr 2023-06-11 21:29:53,808 | server.py:218 | fit_round 4: strategy sampled 18 clients (out of 24)
DEBUG:flwr:fit_round 4: strategy sampled 18 clients (out of 24)


(launch_and_evaluate pid=12582) 1440/1440 - 16s - loss: 0.6147 - tp: 237.0000 - fp: 15108.0000 - tn: 30661.0000 - fn: 74.0000 - accuracy: 0.6705 - precision: 0.0154 - recall: 0.7621 - auc: 0.7670 - prc: 0.0177 - 16s/epoch - 11ms/step
(launch_and_evaluate pid=12582) Writing out interim results.
(launch_and_evaluate pid=12582) Writing out interim results.
(launch_and_fit pid=12578) 
(launch_and_fit pid=12578) This is client:  4
(launch_and_fit pid=12578) Loaded data for client:  4 
(launch_and_fit pid=12578) 
(launch_and_fit pid=12578) Making model:  4
(launch_and_fit pid=12578) Client CID: 4 is done.
(launch_and_fit pid=12578) in fit
(launch_and_fit pid=12578) in server round:  4
(launch_and_fit pid=12578) Epoch 1/3
(launch_and_fit pid=12578) 23/23 - 5s - loss: 0.5803 - tp: 266.0000 - fp: 106.0000 - tn: 253.0000 - fn: 93.0000 - accuracy: 0.7228 - precision: 0.7151 - recall: 0.7409 - auc: 0.7732 - prc: 0.7632 - val_loss: 0.5549 - val_tp: 116.0000 - val_fp: 46.0000 - val_tn: 108.0000 - va

DEBUG flwr 2023-06-11 21:34:37,833 | server.py:232 | fit_round 4 received 18 results and 0 failures
DEBUG:flwr:fit_round 4 received 18 results and 0 failures
DEBUG flwr 2023-06-11 21:34:37,860 | server.py:168 | evaluate_round 4: strategy sampled 19 clients (out of 24)
DEBUG:flwr:evaluate_round 4: strategy sampled 19 clients (out of 24)


(launch_and_fit pid=12578) 60/60 - 3s - loss: 0.6735 - tp: 505.0000 - fp: 311.0000 - tn: 639.0000 - fn: 445.0000 - accuracy: 0.6021 - precision: 0.6189 - recall: 0.5316 - auc: 0.6106 - prc: 0.6103 - val_loss: 0.6381 - val_tp: 442.0000 - val_fp: 231.0000 - val_tn: 434.0000 - val_fn: 223.0000 - val_accuracy: 0.6586 - val_precision: 0.6568 - val_recall: 0.6647 - val_auc: 0.7149 - val_prc: 0.7021 - 3s/epoch - 52ms/step
(launch_and_fit pid=12578) 60/60 - 3s - loss: 0.6735 - tp: 505.0000 - fp: 311.0000 - tn: 639.0000 - fn: 445.0000 - accuracy: 0.6021 - precision: 0.6189 - recall: 0.5316 - auc: 0.6106 - prc: 0.6103 - val_loss: 0.6381 - val_tp: 442.0000 - val_fp: 231.0000 - val_tn: 434.0000 - val_fn: 223.0000 - val_accuracy: 0.6586 - val_precision: 0.6568 - val_recall: 0.6647 - val_auc: 0.7149 - val_prc: 0.7021 - 3s/epoch - 52ms/step
(launch_and_evaluate pid=12578) 
(launch_and_evaluate pid=12578) This is client:  14
(launch_and_evaluate pid=12578) Loaded data for client:  14 
(launch_and_eval

DEBUG flwr 2023-06-11 21:39:14,429 | server.py:182 | evaluate_round 4 received 19 results and 0 failures
DEBUG:flwr:evaluate_round 4 received 19 results and 0 failures
INFO flwr 2023-06-11 21:39:14,434 | server.py:147 | FL finished in 2171.3941023590005
INFO:flwr:FL finished in 2171.3941023590005
INFO flwr 2023-06-11 21:39:14,443 | app.py:218 | app_fit: losses_distributed [(1, 0.722645534614384), (2, 0.6384931632754293), (3, 0.6091061804285216), (4, 0.5963912808842924)]
INFO:flwr:app_fit: losses_distributed [(1, 0.722645534614384), (2, 0.6384931632754293), (3, 0.6091061804285216), (4, 0.5963912808842924)]
INFO flwr 2023-06-11 21:39:14,448 | app.py:219 | app_fit: metrics_distributed_fit {}
INFO:flwr:app_fit: metrics_distributed_fit {}
INFO flwr 2023-06-11 21:39:14,451 | app.py:220 | app_fit: metrics_distributed {'tp': [(1, 400.71824827491974), (2, 306.34491654021247), (3, 339.51132547338335), (4, 326.77665652593305)], 'fp': [(1, 21943.268907563026), (2, 15955.556525037937), (3, 14580.23

In [28]:
# TODO: This cell needs turned into a function.
# Write out the results of evaluation.

count = 0
for key in (fl_history.metrics_distributed):
    print(key)
    results[count][1] = fl_history.metrics_distributed[key][(num_rounds-1):][0][1]
    count += 1
count = 0
results[9][1] = fl_epochs
results[10][1] = num_rounds
results[11][1] = NUM_CLIENTS * 0.75
results[12][1] = NUM_CLIENTS * 0.25

file = results_path + "history" + "_" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S") + ".csv"
results_out = pd.DataFrame(results)
results_out.columns = ['single_server', 'distributed']
results_out.index = ['tp', 'fp', 'tn', 'fn', 'accuracy', 'precision', 'recall', 'auc', 'prc', 'epochs', 'rounds', 'train_clients', 'evaluate_clients']
print(results_out)
pd.DataFrame(results_out).to_csv(file)
print("\nWriting out results.")

tp
fp
tn
fn
accuracy
precision
recall
auc
prc
                  single_server   distributed
tp                   876.000000    326.776657
fp                 10761.000000  13897.033242
tn                 33958.000000  31700.599095
fn                   485.000000    155.591007
accuracy               0.755946      0.695039
precision              0.075277      0.022764
recall                 0.643644      0.696423
auc                    0.766392      0.745412
prc                    0.088764      0.025943
epochs                10.000000      3.000000
rounds                 1.000000      4.000000
train_clients          1.000000     18.000000
evaluate_clients       1.000000      6.000000

Writing out results.


In [29]:
# TODO: Create a function that does all this
# Assemble all of the client round data into one file.
for filename in glob.glob(os.path.join(temp_path, '*.csv')):
    # print("\nnow reading " + filename + "\n")
    # read file
    df = pd.read_csv(filename, index_col=[0])
    client = int(df['0'].iloc[0])
    round = int(df['0'].iloc[1])

    # writing individual interim results to master interim results file
    interim_results[client:client+1, 0][:,round-1] = df['0'].iloc[2]
    interim_results[client:client+1, 1][:,round-1] = df['0'].iloc[3]
    interim_results[client:client+1, 2][:,round-1] = df['0'].iloc[4]
    interim_results[client:client+1, 3][:,round-1] = df['0'].iloc[5]
    interim_results[client:client+1, 4][:,round-1] = df['0'].iloc[6]
    interim_results[client:client+1, 5][:,round-1] = df['0'].iloc[7]
    interim_results[client:client+1, 6][:,round-1] = df['0'].iloc[8]
    interim_results[client:client+1, 7][:,round-1] = df['0'].iloc[9]
    interim_results[client:client+1, 8][:,round-1] = df['0'].iloc[10]

In [30]:

file = results_path + "interim_history" + "_" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S") + ".npy"
# Write consolodated interim history to disk
np.save(file, interim_results)

In [31]:
# Read the array from the file
interim_results_test = np.load(file)
print(interim_results_test)

[[[  210.           187.             0.           179.        ]
  [22866.         15930.             0.         13298.        ]
  [22952.         29888.             0.         32520.        ]
  [   52.            75.             0.            83.        ]
  [    0.50264758     0.65266925     0.             0.70961374]
  [    0.00910036     0.01160266     0.             0.01328189]
  [    0.80152673     0.71374047     0.             0.68320608]
  [    0.72951293     0.73315829     0.             0.74016511]
  [    0.01639433     0.0174455      0.             0.02008218]]

 [[  176.           159.             0.           152.        ]
  [21577.         15360.             0.         13299.        ]
  [24286.         30503.             0.         32564.        ]
  [   41.            58.             0.            65.        ]
  [    0.53085935     0.66540802     0.             0.70998263]
  [    0.00809084     0.01024551     0.             0.01130028]
  [    0.81105989     0.73271888     0

In [32]:
# delete temp files
# https://stackoverflow.com/questions/185936/how-to-delete-the-contents-of-a-folder
folder = temp_path
print("deleting files in: ", folder)
for filename in os.listdir(folder):
    file_path = os.path.join(folder, filename)
    try:
        if os.path.isfile(file_path) or os.path.islink(file_path):
            os.unlink(file_path)
        elif os.path.isdir(file_path):
            shutil.rmtree(file_path)
    except Exception as e:
        print('Failed to delete %s. Reason: %s' % (file_path, e))

deleting files in:  /content/drive/MyDrive/Colab Notebooks/FF/history_temp/
